In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/earthquake-database/database.csv


In [27]:
import numpy as np
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import datetime
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [8]:
# Lets read the CSv file
# Read the CSV file into a DataFrame
df = pd.read_csv("/kaggle/input/earthquake-database/database.csv")

df


,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.2460,145.6160,Earthquake,131.60,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.8630,127.3520,Earthquake,80.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.5790,-173.9720,Earthquake,20.00,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.0760,-23.5570,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.9380,126.4270,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,12/28/2016,08:22:12,38.3917,-118.8941,Earthquake,12.30,1.2,40.0,5.6,ML,...,18.0,42.47,0.120,NaN,0.1898,NN00570710,NN,NN,NN,Reviewed
23408,12/28/2016,09:13:47,38.3777,-118.8957,Earthquake,8.80,2.0,33.0,5.5,ML,...,18.0,48.58,0.129,NaN,0.2187,NN00570744,NN,NN,NN,Reviewed
23409,12/28/2016,12:38:51,36.9179,140.4262,Earthquake,10.00,1.8,NaN,5.9,MWW,...,NaN,91.00,0.992,4.8,1.5200,US10007NAF,US,US,US,Reviewed
23410,12/29/2016,22:30:19,-9.0283,118.6639,Earthquake,79.00,1.8,NaN,6.3,MWW,...,NaN,26.00,3.553,6.0,1.4300,US10007NL0,US,US,US,Reviewed


In [14]:
''' taking only main features '''
df = df[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]
df.head()

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,01/02/1965,13:44:18,19.246,145.616,131.6,6.0
1,01/04/1965,11:29:49,1.863,127.352,80.0,5.8
2,01/05/1965,18:05:58,-20.579,-173.972,20.0,6.2
3,01/08/1965,18:49:43,-59.076,-23.557,15.0,5.8
4,01/09/1965,13:32:50,11.938,126.427,15.0,5.8


Creating Time stamps using Date and Time Columns

In [15]:
time_stamp = []
for d, t in zip(df['Date'], df['Time']):
    try:
        ts = datetime.datetime.strptime(d+' '+t, '%m/%d/%Y %H:%M:%S')
        time_stamp.append(time.mktime(ts.timetuple()))
    except ValueError:
        time_stamp.append('ValueError')

In [17]:
import pandas as pd

# Create a DataFrame named 'data' (assuming you have a DataFrame named 'df')
data = df.copy()  # Copy your existing DataFrame

# Create a Pandas Series for the 'Timestamp' column
time_stamp_series = pd.Series(time_stamp)

# Add the 'Timestamp' Series to the 'data' DataFrame
data['Timestamp'] = time_stamp_series



In [18]:
''' making datafram of timestamp'''
time_stamp = pd.Series(time_stamp)
data['Timestamp'] = time_stamp.values

In [19]:
data

,Date,Time,Latitude,Longitude,Depth,Magnitude,Timestamp
0,01/02/1965,13:44:18,19.2460,145.6160,131.60,6.0,-157630542.0
1,01/04/1965,11:29:49,1.8630,127.3520,80.00,5.8,-157465811.0
2,01/05/1965,18:05:58,-20.5790,-173.9720,20.00,6.2,-157355642.0
3,01/08/1965,18:49:43,-59.0760,-23.5570,15.00,5.8,-157093817.0
4,01/09/1965,13:32:50,11.9380,126.4270,15.00,5.8,-157026430.0
...,...,...,...,...,...,...,...
23407,12/28/2016,08:22:12,38.3917,-118.8941,12.30,5.6,1482913332.0
23408,12/28/2016,09:13:47,38.3777,-118.8957,8.80,5.5,1482916427.0
23409,12/28/2016,12:38:51,36.9179,140.4262,10.00,5.9,1482928731.0
23410,12/29/2016,22:30:19,-9.0283,118.6639,79.00,6.3,1483050619.0


,Latitude,Longitude,Depth,Magnitude
0,19.2460,145.6160,131.60,6.0
1,1.8630,127.3520,80.00,5.8
2,-20.5790,-173.9720,20.00,6.2
3,-59.0760,-23.5570,15.00,5.8
4,11.9380,126.4270,15.00,5.8
...,...,...,...,...
23407,38.3917,-118.8941,12.30,5.6
23408,38.3777,-118.8957,8.80,5.5
23409,36.9179,140.4262,10.00,5.9
23410,-9.0283,118.6639,79.00,6.3


In [20]:
''' dropping Date and Time column '''
data = data.drop(['Date', 'Time'], axis=1)
data = data[data.Timestamp != 'ValueError']  # Use 'data' instead of 'df' for the 'Timestamp' column

In [21]:
data

,Latitude,Longitude,Depth,Magnitude,Timestamp
0,19.2460,145.6160,131.60,6.0,-157630542.0
1,1.8630,127.3520,80.00,5.8,-157465811.0
2,-20.5790,-173.9720,20.00,6.2,-157355642.0
3,-59.0760,-23.5570,15.00,5.8,-157093817.0
4,11.9380,126.4270,15.00,5.8,-157026430.0
...,...,...,...,...,...
23407,38.3917,-118.8941,12.30,5.6,1482913332.0
23408,38.3777,-118.8957,8.80,5.5,1482916427.0
23409,36.9179,140.4262,10.00,5.9,1482928731.0
23410,-9.0283,118.6639,79.00,6.3,1483050619.0


llcrnrlat, urcrnrlat, llcrnrlon, and urcrnrlon define the lower-left corner and upper-right corner latitude and longitude values

In [22]:
X = data[['Timestamp', 'Latitude', 'Longitude']]
y = data[['Magnitude', 'Depth']]


In [23]:
''' converting dtype'''
data['Timestamp'] = data['Timestamp'].astype(float)

In [24]:
data

,Latitude,Longitude,Depth,Magnitude,Timestamp
0,19.2460,145.6160,131.60,6.0,-1.576305e+08
1,1.8630,127.3520,80.00,5.8,-1.574658e+08
2,-20.5790,-173.9720,20.00,6.2,-1.573556e+08
3,-59.0760,-23.5570,15.00,5.8,-1.570938e+08
4,11.9380,126.4270,15.00,5.8,-1.570264e+08
...,...,...,...,...,...
23407,38.3917,-118.8941,12.30,5.6,1.482913e+09
23408,38.3777,-118.8957,8.80,5.5,1.482916e+09
23409,36.9179,140.4262,10.00,5.9,1.482929e+09
23410,-9.0283,118.6639,79.00,6.3,1.483051e+09


In [36]:
X

,Timestamp,Latitude,Longitude
0,-157630542.0,19.2460,145.6160
1,-157465811.0,1.8630,127.3520
2,-157355642.0,-20.5790,-173.9720
3,-157093817.0,-59.0760,-23.5570
4,-157026430.0,11.9380,126.4270
...,...,...,...
23407,1482913332.0,38.3917,-118.8941
23408,1482916427.0,38.3777,-118.8957
23409,1482928731.0,36.9179,140.4262
23410,1483050619.0,-9.0283,118.6639


In [37]:
y

,Magnitude,Depth
0,6.0,131.60
1,5.8,80.00
2,6.2,20.00
3,5.8,15.00
4,5.8,15.00
...,...,...
23407,5.6,12.30
23408,5.5,8.80
23409,5.9,10.00
23410,6.3,79.00


In [29]:
''' train test split '''
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1)

In [31]:
''' RandomForestRegressor '''
rf_reg = RandomForestRegressor(random_state=42)

''' fit on data '''
rf_reg.fit(X_train, y_train)

''' prediction '''
rf_reg.predict(X_test)

array([[  5.861, 509.923],
       [  5.955,  30.586],
       [  5.68 ,  35.862],
       ...,
       [  5.687,  11.957],
       [  5.687,  24.425],
       [  5.556,  46.152]])

In [33]:
''' Score '''
rf_reg.score(X_test, y_test)

0.3651713267506974

In [34]:
''' Linear Regression '''
li_reg = LinearRegression()

''' fit on data '''
li_reg.fit(X_train, y_train)

''' prediction '''
li_reg.predict(X_test)

array([[ 5.89241775, 63.03707992],
       [ 5.90124673, 78.01990077],
       [ 5.8576942 , 97.10635194],
       ...,
       [ 5.90162562, 52.04685846],
       [ 5.8599455 , 94.59043364],
       [ 5.84000464, 95.45894326]])

In [35]:
''' Score '''
li_reg.score(X_test, y_test)

0.002386880734210839